# Bayesian optimization with `skopt`

(based on scikit-optimize documentation https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html)

In [1]:
# sklearn version fixed to avoid known skopt issue
!pip install scikit-optimize scikit-learn==0.20.3 

     |████████████████████████████████| 81kB 28.4MB/s 


In [0]:
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from skopt import BayesSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Optimising a classifier

In [3]:
from sklearn.datasets import load_digits
X, y = load_digits(10, True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75,
                                                    random_state=0)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


dimensions for parameters  [list, shape=(n_dims,)]: List of search space dimensions. Each search dimension can be defined either as

- a (lower_bound, upper_bound) tuple (for Real or Integer dimensions),
- a (lower_bound, upper_bound, prior) tuple (for Real dimensions),
- as a list of categories (for Categorical dimensions), or
- an instance of a Dimension object (Real, Integer or Categorical).

In [0]:
param_dist = {
    "max_depth": (3, 10,),
    "max_features": (1, 11),
    "min_samples_split": <YOUR CODE>, # from 2 to 10
    "min_samples_leaf": <YOUR CODE>, # from 1 to 10
    "bootstrap": [True, False], # categorical valued parameter
    "criterion": <YOUR CODE> # either "gini" or "entropy"
   }

In [0]:
clf = RandomForestClassifier(n_estimators=20)
opt = BayesSearchCV(clf, param_dist, n_iter=10, return_train_score=True, cv=3)

In [0]:
opt.fit(X_train, y_train);

In [7]:
print("val. score: %s" % opt.best_score_)
print("test score: %s" % opt.score(X_test, y_test))

val. score: 0.9465478841870824
test score: 0.9466666666666667


In [0]:
# Utility function to report best scores
import pandas as pd

def report(results, n_top=3):
  res = pd.DataFrame(results)
  res = res.sort_values(by=['mean_test_score'], ascending=False, axis=0)
  res.reset_index(inplace = True, drop=True)
#   a = res[['mean_test_score', 'std_test_score']]

  for candidate in range(0, n_top):
    print("Model with rank: {0}".format(candidate))
    print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
          res['mean_test_score'][candidate],
          res['std_test_score'][candidate]))
    print("Parameters: {0}".format(res['params'][candidate]))
    print("")

In [9]:
report(opt.cv_results_)

Model with rank: 0
Mean validation score: 0.947 (std: 0.003)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 9, 'max_features': 4, 'min_samples_leaf': 3, 'min_samples_split': 10}

Model with rank: 1
Mean validation score: 0.944 (std: 0.006)
Parameters: {'bootstrap': False, 'criterion': 'entropy', 'max_depth': 8, 'max_features': 3, 'min_samples_leaf': 4, 'min_samples_split': 3}

Model with rank: 2
Mean validation score: 0.940 (std: 0.005)
Parameters: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 7, 'min_samples_leaf': 8, 'min_samples_split': 10}



## Task
Optimise the neural net from the previous notebook via `BayesSearchCV` 


In [0]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier
torch.manual_seed(0);

In [0]:
from sklearn.datasets import make_classification
X, y = make_classification(1000, 20, n_informative=10, n_classes=2, random_state=0)
X = X.astype(np.float32)

In [0]:
class ClassifierModule(nn.Module):
<CODE OF THE CLASSIFIER FROM NOTEBOOK i-1>

In [0]:
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    lr=0.1,
    device='cuda',  # comment this to train with CPU
    optimizer__momentum=0.9,
    verbose=0
)

In [0]:
params = {
    'lr': [0.05, 0.1],
    'module__num_units': [10, 20, 30], # range from 10 to 50
    'module__dropout': [0.1, 0.3], # range from 0.1 to 0.3
    'optimizer__nesterov': [False, True],
}

In [0]:
bs = BayesSearchCV(net, params, refit=False, cv=3, scoring='accuracy', 
                  verbose=0, n_jobs=1, n_iter=10, return_train_score=True)

In [0]:
bs.fit(X, y);

In [31]:
report(bs.cv_results_)

Model with rank: 0
Mean validation score: 0.866 (std: 0.016)
Parameters: {'lr': 0.08443659005255347, 'module__dropout': 0.1329029172663855, 'module__num_units': 20, 'optimizer__nesterov': False}

Model with rank: 1
Mean validation score: 0.864 (std: 0.021)
Parameters: {'lr': 0.08294332604267257, 'module__dropout': 0.2614271555754777, 'module__num_units': 30, 'optimizer__nesterov': False}

Model with rank: 2
Mean validation score: 0.859 (std: 0.007)
Parameters: {'lr': 0.06507418901415689, 'module__dropout': 0.15804617348500327, 'module__num_units': 20, 'optimizer__nesterov': True}

